In [158]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/gieunkwak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gieunkwak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### word2vec을 이용한 단어 embedding

In [159]:
# GPT야, simpsons 캐릭터 이름이 들어간 랜덤 문장 10개를 생성해줘

sentences = ["Homer Simpson forgot his lunch at home, so he had to buy a burger on his way to work.",
    "Marge was busy knitting a new sweater for Bart's upcoming school play.",
    "Lisa Simpson played a beautiful saxophone solo at the school concert.",
    "Mr. Burns secretly plotted another scheme from his office at the Springfield Nuclear Power Plant.",
    "Ned Flanders offered to help Homer fix the fence between their houses.",
    "Bart Simpson tried a new prank at school, but it didn't go as planned.",
    "Milhouse and Bart spent the afternoon playing video games and forgot to do their homework.",
    "Maggie Simpson's adorable giggle filled the room as she played with her toys.",
    "Apu had a busy day at the Kwik-E-Mart, dealing with a rush of customers.",
    "Krusty the Clown decided to change his show a bit to attract a new audience."]

In [160]:
# preprocessing
# get rid of stopwords, lower case

sentences = [s.lower().replace(".", "").split(" ") for s in sentences]

In [161]:
sentences[0]

['homer',
 'simpson',
 'forgot',
 'his',
 'lunch',
 'at',
 'home,',
 'so',
 'he',
 'had',
 'to',
 'buy',
 'a',
 'burger',
 'on',
 'his',
 'way',
 'to',
 'work']

In [162]:
# train word2vec

skip_gram = Word2Vec(sentences, vector_size=300, min_count=1, window=5, sg=1)

In [163]:
print("{} 의 vector representation : \n{}".format('homer', skip_gram.wv.get_vector(skip_gram.wv.key_to_index['homer'])))

homer 의 vector representation : 
[ 4.3701060e-04  2.2055381e-03  3.3186206e-03  2.9940126e-03
 -2.6567397e-03  2.1234783e-03 -1.8821131e-03 -2.3819870e-04
  1.6127196e-04  2.1778548e-03  1.5117687e-03  1.5151121e-03
  3.1674108e-03  1.1945881e-04 -2.0241914e-03 -2.1457686e-03
  2.2043383e-03 -1.7624284e-03 -9.4591797e-04  1.2793364e-03
 -7.2408590e-04 -2.0029263e-03 -7.5762207e-04  4.2041132e-04
  7.6597626e-04  2.0379657e-03 -1.7428604e-03  1.0297104e-03
  2.4278299e-03  6.9946179e-04  1.7967316e-03 -1.5996851e-03
  2.0529374e-03 -2.5236835e-03  1.1533408e-03 -3.0791005e-03
 -8.4886025e-04 -3.0566400e-03 -5.1756331e-04 -1.7981926e-03
 -1.2938769e-03  3.8499379e-04  9.4194955e-04 -5.1014154e-04
 -2.6721673e-03 -1.9110956e-03  2.8245136e-04 -1.2844786e-03
 -3.1514806e-03 -2.4119446e-04  2.2119523e-03  1.9877201e-03
 -3.3284628e-03  1.0791372e-03 -2.0474067e-03 -3.0453433e-03
  3.5906611e-05 -8.0902988e-05 -2.3292252e-03 -2.0569263e-03
 -7.9015619e-04  2.3795723e-03 -2.5085832e-03  2.559

In [131]:
skip_gram.wv.most_similar("homer")

[('video', 0.14035673439502716),
 ('his', 0.12365606427192688),
 ('adorable', 0.11183691769838333),
 ('burger', 0.10865367203950882),
 ('planned', 0.09786190837621689),
 ('she', 0.09258976578712463),
 ('do', 0.09069041162729263),
 ('as', 0.08781418949365616),
 ('concert', 0.08753109723329544),
 ('lisa', 0.08640199154615402)]

직접 유사도 구해보기

In [164]:
homer_vector = skip_gram.wv.get_vector(skip_gram.wv.key_to_index['homer'])
video_vector = skip_gram.wv.get_vector(skip_gram.wv.key_to_index['video'])

In [165]:
# 유사도 계산하기 from scratch
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vector_a, vector_b):
    """Calculate the cosine similarity between two vectors."""
    dot_product = np.dot(vector_a, vector_b)
    norm_a = norm(vector_a)
    norm_b = norm(vector_b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [166]:
cosine_similarity(homer_vector, video_vector)

0.14035673

### Simpsons dataset을 활용한 Word2Vec

![](https://images.edrawmax.com/what-is/simpsons-family-tree/example.png) <br>
출처 : https://images.edrawmax.com/what-is/simpsons-family-tree/example.png

In [167]:
import re
import pandas as pd

import spacy

In [168]:
df = pd.read_csv('simpsons_dataset.csv')
df.shape

(158314, 2)

In [169]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [170]:
df.isnull().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

In [172]:
df.loc[0, 'spoken_words']

"No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it."

#### 데이터 전처리

In [173]:
# We are lemmatizing and removing the stopwords and non-alphabetic characters for each line of dialogue.

nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

def cleaning(doc):
    # Lemmatizes and remove stopwords
    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [174]:
# only keep alphabets
cleaner = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

In [175]:
txt = [cleaning(doc) for doc in nlp.pipe(cleaner, batch_size=5000)]

In [176]:
txt[0]

'actually little disease magazine news show natural think'

In [177]:
# dataframe에 넣어서 null이 있는 대화는 삭제
# 주로 null은 특정 행동을 했지만 대화가 없었을 때임

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(85956, 1)

In [178]:
# 하나의 문장을 여러 단위의 단어로 분할
sentences = [s.split(' ') for s in df_clean['clean']]

In [179]:
len(sentences)

85956

In [180]:
sentences[0]

['actually',
 'little',
 'disease',
 'magazine',
 'news',
 'show',
 'natural',
 'think']

#### Word2Vec 모델 훈련

In [181]:
from gensim.models import Word2Vec

In [182]:
help(Word2Vec)

Help on class Word2Vec in module gensim.models.word2vec:

class Word2Vec(gensim.utils.SaveLoad)
 |  Word2Vec(sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), comment=None, max_final_vocab=None, shrink_windows=True)
 |  
 |  Method resolution order:
 |      Word2Vec
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=100

- `window` : 문장 내에서 현재 단어와 예측 단어 사이의 최대 거리. ex) 타겟 단어의 왼쪽과 오른쪽 n번째 단어
- `vector_size` : 단어 벡터의 차원 수
- `min_count` : 이 값보다 총 절대 빈도수가 낮은 모든 단어를 무시함 - (2, 100)
- `sg` : 1은 skip-gram, 0은 CBOW method를 사용

In [183]:
# 모델 정의 하기
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007)

In [184]:
# 문장에 들어있는 각 단어들을 Word2Vec 모델이 인식할 수 있는 형태로 변환
w2v_model.build_vocab(sentences)

In [185]:
# 모델 훈련
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=100)

(19987519, 54001900)

In [187]:
# help(w2v_model.train)

### 단어간 유사도 확인하기

In [189]:
# dir(w2v_model.wv)

- most_similar : 주어진 조건에 가장 적합한 단어 탐색
- similarity : 주어진 단어들의 유사도 계산
- doesnt_match : 주어진 단어들 중 가장 '덜 유사한' 단어

In [191]:
# help(w2v_model.wv.most_similar)

In [105]:
# help(w2v_model.wv.similarity)

In [192]:
w2v_model.wv.most_similar(positive=["homer"])

[('marge', 0.4796435534954071),
 ('simpson', 0.3079630732536316),
 ('bart', 0.2804730534553528),
 ('right', 0.2630842626094818),
 ('husband', 0.25762954354286194),
 ('family', 0.24938347935676575),
 ('lisa', 0.2485237717628479),
 ('dad', 0.24219074845314026),
 ('son', 0.23068201541900635),
 ('mr', 0.22777511179447174)]

In [194]:
w2v_model.wv.most_similar(positive=["bart"])

[('lisa', 0.48629072308540344),
 ('mom', 0.35374078154563904),
 ('boy', 0.31837302446365356),
 ('maggie', 0.31023937463760376),
 ('dad', 0.28740808367729187),
 ('homer', 0.2804730534553528),
 ('think', 0.2786455452442169),
 ('milhouse', 0.2761225700378418),
 ('kid', 0.2718731462955475),
 ('son', 0.2697523832321167)]

In [195]:
w2v_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

[('man', 0.2866703271865845),
 ('people', 0.2453947514295578),
 ('see', 0.24507500231266022)]

In [196]:
w2v_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

[('maggie', 0.3170935809612274),
 ('mom', 0.30980563163757324),
 ('lisa', 0.28484153747558594)]

In [197]:
w2v_model.wv.doesnt_match(['bart', 'homer', 'marge'])

'bart'

In [198]:
w2v_model.wv.doesnt_match(['bart', 'lisa', 'marge'])

'marge'

### 단어 임베딩의 한계점

In [200]:
bank_vector = w2v_model.wv.get_vector(w2v_model.wv.key_to_index['bank'])

- 단어에 불과하기 때문에 context를 고려하지 못 한다

In [201]:
bank_vector

array([ 0.25585392,  0.57371175, -0.06495668, -0.32777452, -1.042156  ,
       -0.50617653,  0.8069787 , -1.4158752 , -0.13325042,  0.17344427,
        0.10152273,  0.561469  ,  0.3041495 , -0.5154144 ,  0.07645763,
        0.12344465, -0.24147335,  0.8387596 ,  0.49487388,  0.8492102 ,
       -0.76920295,  0.04090631, -1.1712912 , -0.40074012,  0.7952797 ,
        0.18193524,  0.7381184 , -1.3339674 ,  0.33916882, -0.57496434,
       -0.6298385 , -0.15693398,  0.47853088,  0.28535807,  0.5132711 ,
        0.44600114,  0.5698779 ,  0.269208  ,  0.01424233,  0.01953911,
        0.5060776 ,  0.12575753, -0.5066727 , -0.9902081 ,  0.525276  ,
        0.75072914, -0.2592828 ,  0.12266743,  0.04692424, -1.21233   ,
       -0.5392707 , -0.10735539, -0.50581706, -0.5784349 ,  0.1565667 ,
        0.45160735,  0.8133438 ,  1.1783154 ,  0.09559247, -0.50142986,
        0.43354177,  0.47021854,  0.19048597, -0.5804716 ,  0.08083846,
       -1.5428402 , -0.37920302, -0.32238775, -0.45251673, -0.14

### sentence embeddings

In [202]:
from transformers import BertTokenizer, BertModel
import torch

In [203]:
# pre-trained model tokenizer와 and bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # smaller & uncased model
model = BertModel.from_pretrained('bert-base-uncased')

In [204]:
# bank가 들어간 유사한 문장 두 개
sentence1 = "I deposited money at the bank."
sentence2 = "The ducks swam to the river bank."

In [205]:
# 문장을 BERT가 인식할 수 있는 형태로 Tokenize
encoded_input1 = tokenizer(sentence1, return_tensors='pt')
encoded_input2 = tokenizer(sentence2, return_tensors='pt')

In [148]:
encoded_input1

{'input_ids': tensor([[  101,  1045, 14140,  2769,  2012,  1996,  2924,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

- `input_ids` : 각 단어별로 매핑된 key. 101은 문장의 시작을, 102는 문장의 끝을 의미
- `token_type_ids` : 문장 번호
- `attention_mask` : attention을 가져야 하는 단어는 1, 그렇지 않은 단어는 0. (만약 input이 실제 단어들이라면 1)

In [206]:
# embedding 생성!
with torch.no_grad():
    output1 = model(**encoded_input1)
    output2 = model(**encoded_input2)

In [207]:
# embedding 내에서 bank라는 단어 찾아오기 (문장의 5번째에 있는 단어)
bank_embedding_sentence1 = output1.last_hidden_state[0, 5, :]
bank_embedding_sentence2 = output2.last_hidden_state[0, 5, :]

In [146]:
# cosine similarity 계산을 통해 얼마나 유사한지 검증

similarity = torch.nn.functional.cosine_similarity(bank_embedding_sentence1, bank_embedding_sentence2, dim=0)

# print("Embedding for 'bank' in sentence 1:", bank_embedding_sentence1)
# print("Embedding for 'bank' in sentence 2:", bank_embedding_sentence2)
print("Cosine similarity between the two embeddings:", similarity)

Cosine similarity between the two embeddings: tensor(0.5922)


--END--